In [2]:
import sys
sys.path.append(r"C:\Users\NLin\OneDrive - AQUILA Commercial\Documents\GitHub\AquilaResearch")
import aquila_tools

import pandas as pd


1. Load data
2. Build Figures
3. Save as HTML String
4. Save to Charts
5. Upload to Github

In [3]:
conn, cursor = aquila_tools.initialize_connection()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
tables = cursor.fetchall()
print("Available tables:", [table[0] for table in tables])


Available tables: ['market_tables_office', 'quarterly_report_data_office', 'issues_stats_industrial', 'issues_stats_office', 'building_list_industrial', 'market_tables_office_survey_overlay', 'quarterly_report_data_industrial', 'v_property_changes_industrial', 'v_property_changes_office', 'inventory_retail', 'key_survey_report_industrial', 'market_tables_industrial', 'building_list_office', 'stats_industrial', 'key_survey_report_office', 'stats_office', 'survey_office', 'survey_industrial', 'inventory_office', 'inventory_industrial', 'audit_log']


In [12]:
import plotly.express as px

from datetime import datetime

# Define a function as a template for consistently styled Plotly charts
def aquila_styled_line_chart(
    df,
    x,
    y,
    color=None,
    facet_row=None,
    title="",
    height=900
):
    # Brand color palette
    aquila_colors = ["#00325a", "#e6b40a", "#8c8c8c", "#18a0CF", "#D05325"]

    fig = px.line(
        df,
        x=x,
        y=y,
        color=color,
        facet_row=facet_row,
        title=title,
        color_discrete_sequence=aquila_colors
    )
    # Add a footer annotation with today's date for "Date Created/Updated"
    today_str = datetime.today().strftime("%Y-%m-%d")
    footer_text = f"Date Created/Updated: {today_str}"

    fig.update_layout(
        height=height,
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(
            family="Futura LT Pro, Futura, Arial, sans-serif",
            color="#00325a"
        ),
        title_font_family="Futura LT Pro, Futura, Arial, sans-serif",
        legend=dict(
            title_font_family="Futura LT Pro, Futura, Arial, sans-serif",
            orientation="h",                # Make legend horizontal (under the chart)
            yanchor="bottom",
            y=-0.25,                        # Place legend well below chart area
            xanchor="center",
            x=0.5
        ),
        margin=dict(b=100),  # increased space at bottom for both annotation and legend
        annotations=[
            dict(
                text=footer_text,
                xref='paper',
                yref='paper',
                x=1,
                y=-0.25,  # place at far bottom right (matches legend y) and more rightward
                showarrow=False,
                font=dict(
                    size=10,
                    family="Futura LT Pro, Futura, Arial, sans-serif",
                    color="#8c8c8c"
                ),
                xanchor='right',
                yanchor='bottom'
            )
        ],
        xaxis_tickangle=90  # Make x-axis tick labels vertical
    )
    return fig

# Example usage of the template:
# Query every row from market_tables_industrial
query = "SELECT * FROM market_tables_industrial;"
df = pd.read_sql(query, conn)

fig = aquila_styled_line_chart(
    df,
    x="quarter",
    y="total_vacancy_rate",
    color="submarket_name",
    facet_row="property_type",
    title="Vacancy Rate by Quarter (Industrial Market) by Submarket and Property Type"
)
fig.show()


C:\Users\NLin\AppData\Local\Temp\ipykernel_34416\124549301.py:73: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [13]:
fig.write_html("charts/vacancy_rate_industrial.html")


Upload to Git

In [18]:
import os
import subprocess

# Make sure we're in the notebook's directory
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
os.chdir(notebook_dir)

# Stage all changes
subprocess.run(["git", "add", "."])

# Commit with a generic message
subprocess.run(["git", "commit", "-m", "Update readme instructions"])

# Push to default remote (origin) and branch (main or master)
# You may want to detect or specify the correct branch
try:
    subprocess.run(["git", "push", "origin", "main"], check=True)
except subprocess.CalledProcessError:
    # If main branch doesn't exist, try master
    subprocess.run(["git", "push", "origin", "master"])

